In [1]:
import pandas as pd
import numpy as np
import os

In [63]:
path = '/Users/rodrigobraga/Downloads/dados_inmet'

yearmonths = sorted([file.split('_')[3].rstrip('.csv') for file in os.listdir(path) if file.startswith('dados')])

## Empilhada em Parquet (not working)

In [ ]:
for year in np.arange(2000,2002):
    df = pd.DataFrame([])
    for month in np.arange(1,12):
        
        tmp = pd.read_csv(path + '/dados_aut_emp_{}.csv'.format(yearmonth), sep='\t', index_col=0)

        tmp['dc_nome'] = tmp['dc_nome'].str.lower()

        tmp['date'] = pd.to_datetime(tmp['dt_medicao'])
        
        df = df.append(tmp)
        
    df.to_csv('../../inmet_emp_{}.csv'.format(year), sep='\t', index=False)
        


## Diária

In [ ]:
df = pd.DataFrame([])

for yearmonth in yearmonths:
    
    print('Processando {}...'.format(yearmonth))
    
    tmp = pd.read_csv(path + '/dados_aut_emp_{}.csv'.format(yearmonth), sep='\t', index_col=0)
    
    tmp['dc_nome'] = tmp['dc_nome'].str.lower()
    
    # create date as datetime

    tmp['date'] = pd.to_datetime(tmp['dt_medicao'])

    # aplica groupby por dia

    tmpg = tmp.groupby([tmp.cd_estacao,
                        tmp.dc_nome,
                        tmp.vl_latitude,
                        tmp.vl_longitude,
                        tmp.date.dt.strftime('%Y-%m-%d')]).\
                        agg(chuva=('chuva','sum'),
                        pre_ins=('pre_ins','mean'),
                        pre_min=('pre_min','min'),
                        pre_max=('pre_max','max'),
                        pto_ins=('pto_ins','mean'),
                        pto_max=('pto_max','max'),
                        pto_min=('pto_min','min'),
                        rad_glo_ins=('rad_glo','mean'),
                        rad_glo_max=('rad_glo','max'),
                        rad_glo_min=('rad_glo','min'),
                        tem_ins=('tem_ins','mean'),
                        tem_max=('tem_max','max'),
                        tem_min=('tem_min','min'),
                        umd_ins=('umd_ins','mean'),
                        umd_max=('umd_max','max'),
                        umd_min=('umd_min','min'),
                        ven_dir_max=('ven_dir','max'),
                        ven_dir_min=('ven_dir','min'),
                        ven_raj_max=('ven_raj','max'),
                        ven_raj_min=('ven_raj','min'),
                        ven_vel_max=('ven_vel','max'),
                        ven_vel_min=('ven_vel','min')).reset_index()

    # convert back do datetime

    tmpg['date'] = pd.to_datetime(tmpg['date'], format='%Y-%m-%d')
    
    df = df.append(tmpg)

In [ ]:
df.to_csv('../../inmet_emp_diaria.csv')

## Mensal

In [ ]:
df = pd.DataFrame([])

for yearmonth in yearmonths:
    
    print('Processando {}...'.format(yearmonth))
    
    tmp = pd.read_csv(path + '/dados_aut_emp_{}.csv'.format(yearmonth), sep='\t', index_col=0)
    
    tmp['dc_nome'] = tmp['dc_nome'].str.lower()
    
    # create date as datetime

    tmp['date'] = pd.to_datetime(tmp['dt_medicao'])

    # aplica groupby por dia

    tmpg = tmp.groupby([tmp.cd_estacao,
                        tmp.dc_nome,
                        tmp.vl_latitude,
                        tmp.vl_longitude,
                        tmp.date.dt.strftime('%Y-%m')]).\
                        agg(chuva=('chuva','sum'),
                        pre_ins=('pre_ins','mean'),
                        pre_min=('pre_min','min'),
                        pre_max=('pre_max','max'),
                        pto_ins=('pto_ins','mean'),
                        pto_max=('pto_max','max'),
                        pto_min=('pto_min','min'),
                        rad_glo_ins=('rad_glo','mean'),
                        rad_glo_max=('rad_glo','max'),
                        rad_glo_min=('rad_glo','min'),
                        tem_ins=('tem_ins','mean'),
                        tem_max=('tem_max','max'),
                        tem_min=('tem_min','min'),
                        umd_ins=('umd_ins','mean'),
                        umd_max=('umd_max','max'),
                        umd_min=('umd_min','min'),
                        ven_dir_max=('ven_dir','max'),
                        ven_dir_min=('ven_dir','min'),
                        ven_raj_max=('ven_raj','max'),
                        ven_raj_min=('ven_raj','min'),
                        ven_vel_max=('ven_vel','max'),
                        ven_vel_min=('ven_vel','min')).reset_index()

    # convert back do datetime

    tmpg['date'] = pd.to_datetime(tmpg['date'], format='%Y-%m-%d')
    
    df = df.append(tmpg)

In [69]:
df.to_csv('../../inmet_emp_mensal.csv')